In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd '/content/drive/MyDrive/mchs/emergency_datahack'

from notebooks.notebook_utils import add_project_path
add_project_path()
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime
import features as f 
from match_station import coord_merge, stations_list, coord_km
from optimizers import optimize
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

/content/drive/MyDrive/mchs/emergency_datahack
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
geo_data = pd.read_csv('./data_raw/geo_data.csv')
# atmos = pd.read_csv('./data_raw/atmos.csv', index_col=0)
tele2 = pd.read_csv('./data_raw/tele2_data.csv', parse_dates=['datetime'])
meteo = pd.read_csv('./data_raw/meteo.csv', index_col=0, parse_dates=['measure_dt'])
# repair = pd.read_csv('./data_raw/repair.csv', parse_dates=['datetime']) 
# crash = pd.read_csv('./data_raw/crash_parts.csv', parse_dates=['datetime'], index_col=0)
traffic = pd.read_csv('./data_raw/traffic.csv', parse_dates=['datetime'])
train = pd.read_csv('./data_raw/train.csv', parse_dates=['datetime'])
test = pd.read_csv('./data_raw/test.csv', parse_dates=['datetime'])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Препроцессинг датасетов с трафиком, погодой и геоданными

In [ ]:
traffic_p = f.preprocess_traffic(traffic)
meteo_p = f.preprocess_meteo(meteo)
geo_data_p = f.preprocess_geo(geo_data, traffic, meteo)
geo_data_p.to_pickle('./data_processed/geo_data_processed.pickle.gz')

In [11]:
geo_data_p = pd.read_pickle('./data_processed/geo_data_processed.pickle.gz')

In [18]:
### Формирование тренировочного датасета

In [ ]:
test_p = f.add_segment_id(test)
test_segment_ids = test_p.segment_id.unique().tolist()

In [ ]:
train_p = f.preprocess_train(train)
train_x = f.compile_train(train_p,
                          start_date='2020-01-01 00:00:00',
                          end_date='2020-12-31 23:00:00',
                          test_segment_ids=test_segment_ids)
train_x.to_pickle('./data_processed/train2020_test_segments.pickle.gz')

### Объединение тренировочного датасета и дополнительных

In [ ]:
train_x = pd.merge(train_x, geo_data_p, on=['road_id', 'road_km'], how='left')

In [ ]:
train_x = pd.merge(train_x, traffic_p, on=['datetime', 'station_id'], how='left')

In [ ]:
train_x = optimize(train_x)

In [ ]:
train_x.drop(columns=['station_id', 'lat_long'], inplace=True)

train_x['lat'] = round(train_x['lat'], 2)
train_x['lon'] = round(train_x['lon'], 2)

In [ ]:
train_x = pd.merge(train_x, meteo_p, on=['datetime', 'meteo_station'], how='left')

In [ ]:
train_x = optimize(train_x)

In [ ]:
train_x.to_pickle('./data_processed/train2020_merge_meteo.pickle.gz')

In [ ]:
train_x = f.add_time_features(train_x)
train_x = optimize(train_x)

In [ ]:
train_x.to_pickle('./data_processed/train2020_merged_time_features.pickle.gz')


In [ ]:
train_x = pd.read_pickle('./data_processed/train2020_merged_time_features.pickle.gz')
train_x.drop(columns=['lat_x', 'lon_x', 'date', 'meteo_station'], inplace=True)
train_x = train_x.rename(columns={'lat_y': 'lat', 'lon_y': 'lon'})
train_x['lat'] = round(train_x['lat'], 2)
train_x['lon'] = round(train_x['lon'], 2)

In [ ]:
train_x = train_x.merge(tele2, on=['datetime', 'road_id', 'road_km'], how='left')

In [ ]:
train_x.to_pickle('./data_processed/train2020_final.pickle.gz')

In [6]:
traffic_cols = ['occupancy_backward', 'occupancy_forward', 
                'speed_backward', 'speed_forward', 
                'volume_backward', 'volume_forward']

traffic_stations = []

for station_id, station_df in traffic_p.groupby(['station_id']):
  for col in traffic_cols:
    station_df[col+'_diff'] = station_df[col].diff()
    station_df[col+'_diff'] = station_df[col+'_diff'].fillna(0)

  traffic_stations.append(station_df)

traffic_p = pd.concat(traffic_stations)

In [8]:
train_x = pd.read_pickle('./data_processed/train2020_final.pickle.gz')

In [12]:
geo_data_p

,road_id,road_km,station_id,dist_nearest_traffic_station,lat_long,lat,lon,meteo_station
0,9,0,55145.0,0.173147,"(55.76, 37.61)",55.759998,37.610001,MOSKMIH
1,9,1,55145.0,0.166630,"(55.76, 37.62)",55.759998,37.619999,MOSKMIH
2,9,2,55145.0,0.155739,"(55.76, 37.63)",55.759998,37.630001,MOSKMIH
3,9,3,55145.0,0.145919,"(55.77, 37.64)",55.770000,37.639999,MOSKMIH
4,9,4,55145.0,0.151824,"(55.77, 37.63)",55.770000,37.630001,MOSKMIH
...,...,...,...,...,...,...,...,...
2811,14,1587,1165.0,1.802952,"(69.61, 30.22)",69.610001,30.219999,NIKEL
2812,14,1588,1165.0,1.799081,"(69.61, 30.22)",69.610001,30.219999,NIKEL
2813,14,1589,1165.0,1.804883,"(69.62, 30.21)",69.620003,30.209999,NIKEL
2814,14,1590,1165.0,1.808937,"(69.63, 30.21)",69.629997,30.209999,NIKEL


In [14]:
traffic_diff_cols = ['occupancy_backward_diff', 'occupancy_forward_diff', 
                    'speed_backward_diff', 'speed_forward_diff', 
                     'volume_backward_diff', 'volume_forward_diff']

train_x = train_x.merge(geo_data_p[['road_id', 'road_km', 'station_id']], on=['road_id', 'road_km'], how='left')
train_x = train_x.merge(traffic_p[['datetime', 'station_id'] + traffic_diff_cols], on=['datetime', 'station_id'], how='left')
train_x = train_x.drop(columns=['station_id'])

In [16]:
train_x[traffic_diff_cols].fillna(0, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [17]:
train_x.to_pickle('./data_processed/train2020_final_2.pickle.gz')

In [5]:
train_x = pd.read_pickle('./data_processed/train2020_final_2.pickle.gz')

In [6]:
hist_counter = train[train.road_id!=5]
hist_counter = hist_counter.groupby(['road_id', 'road_km']).target.sum().reset_index()
hist_counter = hist_counter.rename(columns={'target': 'historical_counter'})

In [7]:
train_x = train_x.merge(hist_counter, on=['road_id', 'road_km'], how='left')

In [ ]:
train_x.to_pickle('./data_processed/train2020_final_3.pickle.gz')

In [9]:
train_x = f.add_segment_id(train_x)

In [10]:
train_x.to_pickle('./data_processed/train2020_final_4.pickle.gz')